In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [3]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) # Normalize the data
train_dataset = datasets.MNIST(root='./dataset_mnist', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./dataset_mnist', train=False, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

"""
train_dataset = datasets.MNIST(root="dataset_mnist/", train=True, transform=transforms.ToTensor(), download = True)
test_dataset = datasets.MNIST(root="dataset_mnist/", train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
"""

'\ntrain_dataset = datasets.MNIST(root="dataset_mnist/", train=True, transform=transforms.ToTensor(), download = True)\ntest_dataset = datasets.MNIST(root="dataset_mnist/", train=False, transform=transforms.ToTensor(), download=True)\n\ntrain_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)\ntest_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)\n'

In [4]:
# Initialize the model, loss function and optimizer
model = SimpleNN(input_size=784, hidden_size=500, num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

print(model)

SimpleNN(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [6]:
total_steps = len(train_loader)
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_steps, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.3479
Epoch [1/5], Step [200/600], Loss: 0.1157
Epoch [1/5], Step [300/600], Loss: 0.3476
Epoch [1/5], Step [400/600], Loss: 0.4425
Epoch [1/5], Step [500/600], Loss: 1.1320
Epoch [1/5], Step [600/600], Loss: 1.0208
Epoch [2/5], Step [100/600], Loss: 0.7147


KeyboardInterrupt: 

In [ ]:
# Testing the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))